In [2]:
!pip install qubovert

In [1]:
import pyqubo as pq
import numpy as np
import qubovert as qv

In [2]:
def chi(precision, number):
    # number must be between 0 and 2
    q = np.zeros((number.size, precision))
    print(q)
    for i in range(len(number)):
        for r in range(precision):
            if number[i] < 2**(-r):
                q[i,r]=0
            else:
                q[i,r]=1
                number[i] -= 2**(-r)
    return q

test = np.array([.126,.6721,1.46])
chi(5, test)




[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


array([[0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0.],
       [1., 0., 1., 1., 1.]])

In [180]:
def ConvertSolution_old(q, a, b):
    '''
    Function to convert an array of qubits into a real number 
    '''
    
    # Store length of q vector
    n = np.shape(q)[0]
    
    # Initialise chi
    chi = 0
    
    for i in range(n):
        chi += (2 ** (-i)) * q[i]
    #print('.',chi)
    
    gamma = (b - a)/2
    x = gamma * chi - 1
    
    return x

def ConvertSolution(q, a, b):
    '''
    Vectorised function to convert an array of an array of qubits into an array of real numbers
    '''
    q = np.array(q)
    
    if len(np.shape(q))==1: # Backwards compatibility
        q = np.array([q])
        #return ConvertSolution_old(q,a,b)
    
    # Convert to binary
    shape = q.shape # shape
    powers = 2.**np.arange(0,-shape[-1],-1) # powers of 2 in descending
    chi = np.sum(q*powers,axis=1) # multiply in
    #print('.',chi)
    
    # Scale and shift
    gamma = (b - a)/2
    #print(gamma)
    x = gamma * chi + a
    
    return x

    
    
#ConvertSolution([0, 1, 1, 1, 1, 0, 1, 1, 0, 0], -1, 3)
print(ConvertSolution_old([1,1,1,0,1], -8, 8))
ConvertSolution([1,1,1,0,1], -8, 8)

13.5


array([6.5])

In [344]:


def H_old(M, y, R = 2, a = -1, b = 3):
    '''
    M and y come from problem Mx-y
    R is bit-precision, ie number of qubits
    a, b endpoints of desired range, a < b, [a,b)
    
    https://arxiv.org/pdf/1901.06526.pdf Section 3
    '''
    
    # Think I need Q to be upper triangular, how best to do this?
    # Need to add in the multiplication by pho 🍜 part as well!!
    
    
    # Save the dimension of problem as N
    N = np.shape(y)[0]

    # Define the midpoint of the range, hard coded atm
    gamma = (b - a)/2
    pho = gamma ** 2
    #print(gamma,pho)
    print('Precision:',(b-a)/2**R)
    
    #Intialise Q as vector of zeros 
    Q = np.zeros((N * R, N * R))
    
    # First term 
    for i in range(N):
        #print(f'i is {i}')
        for r in range(R):
            #print(f'r is {r}')
            for j in range(N):
                # print(f'j is {j}')
                for s in range(R):
                    # print(f's is {s}')
                    for k in range(N):
                        # print(f'k is {k}')
                        
                        # Update relevant entry of Q
                        Q[i * R + r,j * R + s] += (2 ** (-r-s)) * M[k,i] * M[k,j] * pho
                        
                        
    # Second term
    # Loop over each number 
    for i in range(N):
        # Loop over each qubit encoding said number
        for r in range(R):
            # Loop of k for matrix multiplication in index form
            for k in range(N):
                Q[i * R + r, i * R + r] += (2 ** (-r)) * M[k,i] * M[k,i] * -2 * gamma 
                
    # Third term
    # Loop over each number
    for i in range(N):
        # Loop over j for the matrix multiplication
        for j in range(N):
            # Loop over each qubit encoding the number
            for r in range(R):
                Q[i * R + r, i * R + r] +=  (2 ** (-r)) * M[j,i] * y[j]  * -2 * gamma 
                
                
#     # Fourth term (necessary?)
#     for i in range(N):
#         for j in range(N):
#             Q
    
    # Make Q upper triangular
    return Q+np.triu(Q)-np.tril(Q)

A = np.array([[2,-1],[3,1]])
B = np.array([3,2]) #soln = 1,-1


a_test = -16
b_test = 32+a_test # b-a needs to be power of 2
accuracy = 5
Q = H_old(A,B, R = accuracy, a = a_test, b = b_test)
Q_qubo = qv.utils.matrix_to_qubo(Q)
print(Q)
#Q_qubo.solve_bruteforce()

state = qv.sim.anneal_qubo(Q_qubo, num_anneals = 100*2**accuracy)
res = list(state.best.state.values())
print(res)
x1 = res[:accuracy]
x2 = res[accuracy:accuracy*2]
print(x1,x2)
print(ConvertSolution(x1,a_test,b_test))
print(ConvertSolution(x2,a_test,b_test))

Precision: 1.0
[[ 2.528e+03  3.328e+03  1.664e+03  8.320e+02  4.160e+02  5.120e+02
   2.560e+02  1.280e+02  6.400e+01  3.200e+01]
 [ 0.000e+00  4.320e+02  8.320e+02  4.160e+02  2.080e+02  2.560e+02
   1.280e+02  6.400e+01  3.200e+01  1.600e+01]
 [ 0.000e+00  0.000e+00  8.000e+00  2.080e+02  1.040e+02  1.280e+02
   6.400e+01  3.200e+01  1.600e+01  8.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -4.800e+01  5.200e+01  6.400e+01
   3.200e+01  1.600e+01  8.000e+00  4.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00 -3.700e+01  3.200e+01
   1.600e+01  8.000e+00  4.000e+00  2.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  4.800e+02
   5.120e+02  2.560e+02  1.280e+02  6.400e+01]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   1.120e+02  1.280e+02  6.400e+01  3.200e+01]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  2.400e+01  3.200e+01  1.600e+01]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+

In [380]:
def anneal(A,B,accuracy,a,b):
    '''Outputs solution to Ax=b''' 
    # Err catch
    assert a<b, 'a needs to be less than b'

    # Calculation
    Q = H(A,B, R = accuracy, a = a, b = b) # Get Q
    Q_qubo = qv.utils.matrix_to_qubo(Q) # Convert to qubo
    state = qv.sim.anneal_qubo(Q_qubo, num_anneals = 100*2**accuracy) # Pass to anneal
    res = np.array(list(state.best.state.values())) # Result
    x = res.reshape((len(res)//accuracy,accuracy)) # Split result for each component
    print(x)
    sol = ConvertSolution(x,a,b) # Convert each component to decimal
    return sol


A = np.array([[2,-1],[3,1]])
B = np.array([3,2]) #soln = 1,-1
sol = anneal(A,B,5,-16,16)
print(sol)

16.0 256.0
Precision: 1.0
[[-5.200000e+01  2.600000e+01  1.300000e+01  6.500000e+00  3.250000e+00
   4.000000e+00  2.000000e+00  1.000000e+00  5.000000e-01  2.500000e-01]
 [ 2.600000e+01 -3.900000e+01  6.500000e+00  3.250000e+00  1.625000e+00
   2.000000e+00  1.000000e+00  5.000000e-01  2.500000e-01  1.250000e-01]
 [ 1.300000e+01  6.500000e+00 -2.275000e+01  1.625000e+00  8.125000e-01
   1.000000e+00  5.000000e-01  2.500000e-01  1.250000e-01  6.250000e-02]
 [ 6.500000e+00  3.250000e+00  1.625000e+00 -1.218750e+01  4.062500e-01
   5.000000e-01  2.500000e-01  1.250000e-01  6.250000e-02  3.125000e-02]
 [ 3.250000e+00  1.625000e+00  8.125000e-01  4.062500e-01 -6.296875e+00
   2.500000e-01  1.250000e-01  6.250000e-02  3.125000e-02  1.562500e-02]
 [ 4.000000e+00  2.000000e+00  1.000000e+00  5.000000e-01  2.500000e-01
   0.000000e+00  4.000000e+00  2.000000e+00  1.000000e+00  5.000000e-01]
 [ 2.000000e+00  1.000000e+00  5.000000e-01  2.500000e-01  1.250000e-01
   4.000000e+00 -2.000000e+00  1

In [381]:
A = np.array([[2,-1],[3,1]])
A = np.eye(2)
C = np.array([2,-2]) 
B = A@C
sol = anneal(A,B,5,-16,16)
print(sol)

16.0 256.0
Precision: 1.0
[[-8.        2.        1.        0.5       0.25      0.        0.
   0.        0.        0.      ]
 [ 2.       -5.        0.5       0.25      0.125     0.        0.
   0.        0.        0.      ]
 [ 1.        0.5      -2.75      0.125     0.0625    0.        0.
   0.        0.        0.      ]
 [ 0.5       0.25      0.125    -1.4375    0.03125   0.        0.
   0.        0.        0.      ]
 [ 0.25      0.125     0.0625    0.03125  -0.734375  0.        0.
   0.        0.        0.      ]
 [ 0.        0.        0.        0.        0.        8.        2.
   1.        0.5       0.25    ]
 [ 0.        0.        0.        0.        0.        2.        3.
   0.5       0.25      0.125   ]
 [ 0.        0.        0.        0.        0.        1.        0.5
   1.25      0.125     0.0625  ]
 [ 0.        0.        0.        0.        0.        0.5       0.25
   0.125     0.5625    0.03125 ]
 [ 0.        0.        0.        0.        0.        0.25      0.125
   0.0625  

In [451]:
# assert 1==0, 'dont run below'

def H(M, y, R = 2, a = -1, b = 3):
    '''
    M and y come from problem Mx-y
    R is bit-precision, ie number of qubits
    a, b endpoints of desired range, a < b, [a,b)
    
    https://arxiv.org/pdf/1901.06526.pdf Section 3, eq 3.17-18
    ''' 
    
    # Save the dimension of problem as N
    N = np.shape(y)[0]

    # Define the midpoint of the range, hard coded atm
    gamma = (b - a)/2
    pho = gamma ** 2 # 🍜
    print(gamma,pho)
    print('Precision:',(b-a)/2**R, f'Range: [{a},{b})')
    
    #Intialise Q as vector of zeros 
    Q = np.zeros((N * R, N * R))
    
    # Compute strengths
    MTM = M.T@M
    
    strengths = np.zeros_like(Q)
    for r in range(R):
        for s in range(R): # nb. includes r=s, which is used for weights
            block = pho * 2**(-r-s) * MTM
            ind_s = np.arange(N)*R + s
            ind_r = np.arange(N)*R + r
            strengths[np.ix_(ind_s, ind_r)] += np.copy(block)
    Q += np.copy(strengths)
    
    
    # Compute weights
    weights = np.zeros_like(Q)
    
    My = np.diag(M.T@y)
    sumMTM = np.diag(np.sum(MTM, axis=1))
    for r in range(R):
        ind_r = np.arange(N)*R + r
        block1 = 2*gamma *2**(-r) * -1 * My
        block2 = -a*2*gamma *2**(-r) * -1 * sumMTM
        weights[np.ix_(ind_r, ind_r)] += np.copy(block1) + np.copy(block2)
    Q += np.copy(weights)
 

    
    
    # Make Q upper triangular
    print(Q)
    return Q+np.triu(Q)-np.tril(Q)

A = np.array([[2,-1],[3,1]])
#A = np.eye(2)
C = np.array([4,-8]) 
B = A@C

a=-16
b=16
R=4

sol = anneal(A,B,R,a,b)
print(sol)


16.0 256.0
Precision: 2.0 Range: [-16,16)
[[-5.248e+03  1.664e+03  8.320e+02  4.160e+02  2.560e+02  1.280e+02
   6.400e+01  3.200e+01]
 [ 1.664e+03 -3.456e+03  4.160e+02  2.080e+02  1.280e+02  6.400e+01
   3.200e+01  1.600e+01]
 [ 8.320e+02  4.160e+02 -1.936e+03  1.040e+02  6.400e+01  3.200e+01
   1.600e+01  8.000e+00]
 [ 4.160e+02  2.080e+02  1.040e+02 -1.020e+03  3.200e+01  1.600e+01
   8.000e+00  4.000e+00]
 [ 2.560e+02  1.280e+02  6.400e+01  3.200e+01 -6.400e+02  2.560e+02
   1.280e+02  6.400e+01]
 [ 1.280e+02  6.400e+01  3.200e+01  1.600e+01  2.560e+02 -4.480e+02
   6.400e+01  3.200e+01]
 [ 6.400e+01  3.200e+01  1.600e+01  8.000e+00  1.280e+02  6.400e+01
  -2.560e+02  1.600e+01]
 [ 3.200e+01  1.600e+01  8.000e+00  4.000e+00  6.400e+01  3.200e+01
   1.600e+01 -1.360e+02]]
[[1 0 1 0]
 [0 1 0 0]]
[ 4. -8.]


In [427]:
Q = H_old(A,B, R,a,b)
print(Q)
Q_qubo = qv.utils.matrix_to_qubo(Q)

state = qv.sim.anneal_qubo(Q_qubo, num_anneals = 100*2**accuracy)
res = list(state.best.state.values())
print(res)
x1 = res[:4]
x2 = res[4:]
print(x1,x2)
print(ConvertSolution(x1,a,b))
print(ConvertSolution(x2,a,b))

Precision: 1.0
[[800. 832. 416. 208. 128.  64.  32.  16.]
 [  0. 192. 208. 104.  64.  32.  16.   8.]
 [  0.   0.  44.  52.  32.  16.   8.   4.]
 [  0.   0.   0.   9.  16.   8.   4.   2.]
 [  0.   0.   0.   0.  48. 128.  64.  32.]
 [  0.   0.   0.   0.   0.  -8.  32.  16.]
 [  0.   0.   0.   0.   0.   0. -12.   8.]
 [  0.   0.   0.   0.   0.   0.   0.  -8.]]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0] [0, 0, 1, 0]
[-8.]
[-6.]


array([3, 2])

In [167]:
a_test = -16
b_test = 32+a_test # b-a needs to be power of 2
accuracy = 5
Q = H(A,B, R = accuracy, a = a_test, b = b_test)
Q_qubo = qv.utils.matrix_to_qubo(Q)
#print(Q)
#Q_qubo.solve_bruteforce()

state = qv.sim.anneal_qubo(Q_qubo, num_anneals = 100*2**accuracy)
res = list(state.best.state.values())
print(res)
x1 = res[:accuracy]
x2 = res[accuracy:accuracy*2]
print(x1,x2)
print(ConvertSolution_old(x1,a_test,b_test))
print(ConvertSolution_old(x2,a_test,b_test))

Precision: 1.0
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1] [0, 0, 0, 0, 0]
2.0
-1.0


In [ ]:

    
    
    
#     strengths = np.empty((N*R, N*R))
#     for r in range(R):
#         sub_block = np.empty((R, R))
#         if r>0:
#             sub_block = np.concatenate([sub_block, np.zeros((R,R*r))], axis=1)
#         for s in range(R):
#             block = 4 * 2**(-r-s) * MTM
#             print(sub_block,'\n')
#             sub_block = np.concatenate([sub_block, block], axis=1)
#         print(strengths,'\n')
#         strengths = np.concatenate([sub_block, block], axis=0)
#     print(strengths)
    
    

In [10]:
A = np.array([[2, -1], [3, 1]])
b = np.array([3,2])

Q = H(2,2, A, b)
print(Q)

qubo = qv.utils.matrix_to_qubo(Q)1
print(qubo.pretty_str())

qubo.solve_bruteforce()

res = qv.sim.anneal_qubo(qv.utils.matrix_to_qubo(Q), num_anneals=10)

print(res.best)

IndexError: tuple index out of range

In [9]:
A = np.array([[-7, -2], [6, 3]])
b = np.array([-4,-5])

Q = get_Q(3,2, A, b)
print(Q)
qubo = qv.utils.matrix_to_qubo(Q)
print(qubo.pretty_str())

print(qubo.solve_bruteforce())

res = qv.sim.anneal_qubo(qv.utils.matrix_to_qubo(Q), num_anneals=10)

print(res.best)

NameError: name 'get_Q' is not defined